## Getting started with TensorFlow and machine learning
## 텐서플로우와 머신러닝 시작하기

#### Brief words about machine learning and tensorflow
많은 데이터 있고 그것을 학습하고 싶을 때, 머신러닝이 필요하다.
데이터는 보통 입력과 출력으로 주어지고, 컴퓨터는 입력과 출력 사이의 패턴을 찾기를 원한다.
간단한 예로 그램과 킬로그램 사이의 관계같은 입력과 출력 사이의 간단한 관계가 있을수 있다. 이 관계는 선형함수로 표현할 수 있다.
그러나, 머신러닝은 입력과 출력 사이의 관계가 첫보기에 불분명할 때 좀 더 재밌고 파워풀해진다.
이 튜토리얼에서 예를 들어, 숫자 이미지와 숫자 이미지의 레이블을 학습할 것이다.
텐서플로우 웹사이트에 따르면, 이 예제는 머신러닝의 "Hello world"이다. 시작해보자.

#### Methodology and approach to the problem
머신러닝 문제는 4개의 부분으로 나눌 수 있다
* 데이터 얻기
* 모델 명세(입력과 출력 사이의 관계)
* 탐험해야 할 모델의 집합 중 제일 좋은 모델을 찾기 위해 컴퓨터 학습시키기
* 모델이 새로운 데이터의 출력을 잘 예측하는지 평가하기

##### The data
처음이자 가장 중요한 부분은 초기 데이터이다.
이 데이터는 컴퓨터가 입력과 출력 사이의 패턴을 어떻게 찾는지 자신을 학습시키기 위한 학습자료이다.
대부분의 경우, 데이터는 깨끗한 포맷으로 주어지지 않는다. 그래서 컴퓨터가 무엇을 보는지(피쳐)와 무엇을 예측하는지 알게 하기 위해 전처리가 필요하다.
머신러닝에서 중요한 도전은 데이터에서 예측력이 가장 좋은 피쳐를 추출하는 것이다.
이 분야는 피쳐 엔지니어닝이라고 불리고 머신러닝 기술의 가장 중요한 요소이다.
최근에, 딥러닝과 같은 머신러닝의 형태는 값비싼 인간의 피쳐 엔지니어닝 노력을 감소시키고 날 데이터로부터 복잡한 출력을 학습할 수 있게 해준다.
이 예제에서, 우리가 볼 데이터는 MNIST이다. MNIST는 흑백의 숫자 이미지(28*28 픽셀 행렬)와 숫자 이미지에 대한 레이블(숫자이미지가 표현하는 숫자)이 달려있다.
첫 단계는 저장소에서 데이터 셋을 다운로드 받는것이다.
다행이, 이 일을 해주는 코드가 이미 다음 세줄의 코드로 작성되어 있다.

In [8]:
import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print "Hey"

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Hey


데이터 다운로드 후, 입력과 출력 페어로 나누길 원한다.
28*28 흑백이미지를 표현하는 가장  간단한 방법은 784 픽셀 값의 리스트로 표현하는 것이다.
출력으로, 각 숫자는 one_hot 벡터로 표현할 수 있다.(숫자가 i이면 i번째 원소가 1)
예를 들어 3은 [0,0,0,1,0,0,0,0,0,0] 와 같은 모양이다.

In [9]:
image_vector = tf.placeholder("float", [None, 784]) 
label = tf.placeholder("float", [None, 10])

코드의 이전 두 줄에서, 텐서플로우한테 데이터의 입력과 레이블이 784개의 float 타입의 피쳐벡터와 10차원의 one-hot 벡터로 올 것임을 알려준다.

#### Modelling part 1 : Choosing the model
데이터를 다운로드 받은 후, 컴퓨터가 찾을 이미지 벡터와 레이블 사이의 관계의 타입을 정의한다.
이 예에서는 모델은 로지스틱 리그레션을 쓴다.
로지스틱 리그레션에서, 모든 숫자 레이블에 대해서 피쳐벡터의 각 피쳐의 가중치 합을 한다.
이 합은 각 한 예제가 클래스에 속하는지에 대해서 컴퓨터가 얼마나 믿는지에 대한 지표이다.
로지스틱 리그레션 모델의 출력은 레이블 클래스의 신뢰도 점수로 볼수 있다.
예를 들어,  [0.2,0,0.8,0,0,0,0,0,0,0] 의 출력은 이미지는 2라는 확신은 80%, 0이라는 확신은 20%라는 것이다.

In [10]:
Weights = tf.Variable(tf.zeros([784, 10]))
bias = tf.Variable(tf.zeros([10]))
confidence_score = tf.nn.softmax(tf.matmul(image_vector, Weights) + bias) 
print "Model chosen"

Model chosen


#### Modelling part 2  : Choosing the loss function 
입력과 출력 사이의 관계를 찾는 모델을 정의한 후, 모델이 얼마나 옳고 얼마나 틀린지 측정하는 방법을 찾기를 원했다.
이 예제에서는, 크로스-엔트로피 함수를 사용한다.

In [11]:
learning_rate = 0.02
loss_function = -tf.reduce_sum(label*tf.log(confidence_score)) 
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function) 
print "Loss function chosen"

Loss function chosen


손실함수를 정의한 후, 학습 프로세스의 목표는 손실함수를 최소화하는 가중치와 바이어스의 집합을 학습하는 것이라고 프로그램에게 말할 수 있다.

#### Training : Showing labeled examples to the computer 
모든 변수를 정의한 후, 변수에 난수값을 초기화하고 훈련 과정의 부분은 이 초기화값을 수정한다.
예제를 보여주는 것을 시작하기 위해, 텐서플로우 세션을 연다.
텐서플로우 세션은 주어진 리소스의 집합으로 텐서플로우 연산들을 미리 정의할 수 있게 해준다.
머신러닝은 컴퓨팅-인텐시브하기 때문에, 리소스들을 알맞게 할당하고, 모든 계산을 수행한 후 리소스를 반환하기를 원한다.

In [12]:
init = tf.initialize_all_variables()
print "Done initializing"

Done initializing


In [13]:
training_epochs = 1000
mini_batch_size = 30
display_step=1
print "Training parameters chosen"

Training parameters chosen


computation graph를 시작한다

In [ ]:
# Launch the graph
print "Hey"
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        print epoch
        avg_loss = 0.
        total_batch_size = int(mnist.train.num_examples/mini_batch_size)
        # Loop over all batches
        for i in range(total_batch_size):
            batch_images, batch_labels = mnist.train.next_batch(mini_batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={image_vector: batch_images, label: batch_labels})
            # Compute average loss
            avg_loss += sess.run(loss_function, feed_dict={image_vector: batch_images, label: batch_labels})/total_batch_size
#         # Display logs per epoch step
#         if epoch % display_step == 0:
#             print "At epoch:", '%04d' % (epoch+1), "the average loss is", "{:.9f}".format(avg_loss)

    print "Training done"
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({image_vector: mnist.test.images, labels: mnist.test.labels})

Hey
0
1
2
3
